In [10]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd

In [11]:
mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic

Initial feed and marking landmarks

In [16]:
cap=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,frame=cap.read()
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable=False


        results=holistic.process(image)


        image.flags.writeable=True
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION,mp_drawing.DrawingSpec(color=(0,0,0),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1))

        mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1))
        
        mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1))
        
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1))


        cv2.imshow("feed",image)
        
        if cv2.waitKey(100) & 0xFF==ord("q"):
            break
    cap.release()
    cv2.destroyAllWindows()

In [19]:
results.face_landmarks.landmark

[x: 0.6159348487854004
y: 0.567969799041748
z: -0.026274049654603004
, x: 0.6177976727485657
y: 0.5167912840843201
z: -0.06531573086977005
, x: 0.6154943704605103
y: 0.5286455750465393
z: -0.031045852228999138
, x: 0.6030992269515991
y: 0.45540758967399597
z: -0.05571819096803665
, x: 0.6175370216369629
y: 0.4997382164001465
z: -0.07124581187963486
, x: 0.616134762763977
y: 0.47449976205825806
z: -0.06882891803979874
, x: 0.6118662357330322
y: 0.41031235456466675
z: -0.04197919741272926
, x: 0.5170829892158508
y: 0.4048781394958496
z: 0.0010003759525716305
, x: 0.6094516515731812
y: 0.3605804145336151
z: -0.038063377141952515
, x: 0.6087799072265625
y: 0.33628374338150024
z: -0.043231330811977386
, x: 0.6054700613021851
y: 0.25845587253570557
z: -0.041392501443624496
, x: 0.6159158945083618
y: 0.5761057734489441
z: -0.023132726550102234
, x: 0.6157508492469788
y: 0.5816311836242676
z: -0.017992736771702766
, x: 0.6154628992080688
y: 0.5830581188201904
z: -0.011844385415315628
, x: 0.61